In [1]:
# %pip install 'accelerate>=0.26.0'
# %pip install torch
# %pip install transformers
# %pip install accelerate
# %pip install -U transformers bitsandbytes
# !python -m pip install optimum


In [1]:
import torch
from transformers import pipeline
import json

# Initialize pipeline
pipe = pipeline(
    task="text-generation",
    model="Qwen/Qwen2.5-1.5B-Instruct",
    max_new_tokens=256,  # Reduce token count
    device_map="mps",  # Use Metal Performance Shaders
    torch_dtype=torch.bfloat16,  # Lower precision
    model_kwargs={"use_cache": True}
)

pipe.model = pipe.model.to(torch.bfloat16)
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token

def save_conversation(chat_history, filename="cooking_session.json"):
    with open(filename, 'w') as f:
        json.dump(chat_history, f, indent=2)
    print(f"Saved to {filename}")

def cook_recipe(recipe_data, automated_inputs):
    # Initialize fresh conversation
    chat = [
        {
            "role": "system", 
            "content": "You are a master chef. Guide through recipe steps sequentially. Start with asking if the user is ready to cook for proceeding to step 1 of the recipe provided."
        }
    ]
    
    # Structured recipe submission
    chat.append({
        "role": "user",
        "content": f"""Guide me through this recipe step-by-step:
        
        TITLE: {recipe_data['title']}
        
        INGREDIENTS:
        {chr(10).join(recipe_data['ingredients'])}
        
        INSTRUCTIONS:
        {recipe_data['instructions']}"""
    })
    
    # Process automated inputs
    for user_input in automated_inputs:
        # Generate response
        outputs = pipe(chat)
        chef_response = outputs[0]["generated_text"][-1]["content"]
        print(f"\nChef: {chef_response}")
        chat.append({"role": "system", "content": chef_response})

        # Add automated input
        print(f"\nTrainee: {user_input}")
        chat.append({"role": "user", "content": user_input})

        if "exit" in user_input.lower():
            break
    
    save_conversation(chat)
    return chat

# Recipe data structure
recipe = {
    "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
        '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
        '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
    ],
    "instructions": """
        1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.

        2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.

        3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.

        4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.
        Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.

        5. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.

        6. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.

        7. Using your fingers, mash flour and butter in a small bowl to combine.

        8. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.

        9. Serve chicken with gravy and squash panzanella alongside.
            """
}

# Predefined automated trainee inputs
automated_inputs = [
    "Got it! Ready for step 1.",
    "Done Continue. 1",
    "Done Continue. 2",
    "Done Continue. 3",
    "Done Continue. 4"
    "Done Continue. 5",
    "Done Continue. 6",
    "Done Continue. 7",
    "Done Continue. 8",
    # "Finished step 2! What's next?",
    # "I need clarification on peeling the squash.",
    # "Okay to proceed.",
    "exit"
]

# Start cooking session with automated inputs
cook_recipe(recipe, automated_inputs)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use mps
/Users/adamoentoro/miniconda3/lib/python3.12/site-packages/transformers/pytorch_utils.py:328: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)



Chef: Are you ready to start cooking?

Trainee: Got it! Ready for step 1.

Chef: Great! For step 1 of our Miso-Butter Roast Chicken With Acorn Squash Panzanella recipe, we need to prepare the chicken. Here’s how we proceed:

### Step 1: Preparing the Chicken

**Ingredients Needed:**
- A whole chicken
- Kosher salt
- Freshly ground black pepper

**Instructions:**

1. **Pat Dry:** Take your whole chicken and pat it dry with paper towels. This helps ensure that the chicken cooks evenly without any moisture interfering with the seasoning process.

2. **Seasoning:** Season the chicken all over with 2 teaspoons of kosher salt. Remember to sprinkle some additional salt around the chicken's surface before tying the legs together.

3. **Tying Legs Together:** Tie the legs of the chicken together using kitchen twine. This helps hold the leg joints tight during roasting, ensuring even cooking throughout the bird.

4. **Let Sit:** Allow the seasoned chicken to sit at room temperature for one hour

[{'role': 'system',
  'content': 'You are a master chef. Guide through recipe steps sequentially. Start with asking if the user is ready to cook for proceeding to step 1 of the recipe provided.'},
 {'role': 'user',
  'content': 'Guide me through this recipe step-by-step:\n        \n        TITLE: Miso-Butter Roast Chicken With Acorn Squash Panzanella\n        \n        INGREDIENTS:\n        1 (3½–4-lb.) whole chicken\n2¾ tsp. kosher salt, divided, plus more\n2 small acorn squash (about 3 lb. total)\n2 Tbsp. finely chopped sage\n1 Tbsp. finely chopped rosemary\n6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature\n¼ tsp. ground allspice\nPinch of crushed red pepper flakes\nFreshly ground black pepper\n⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)\n2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces\n2 Tbsp. extra-virgin olive oil\n½ small red onion, thinly sliced\n3 Tbsp. apple cider vinegar\n1 Tbsp. white mi